In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# English Translated

In [ ]:
data=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df_en_processed.pkl').reset_index()#.drop_duplicates('clean_msg')
temp=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df.pkl').reset_index()
data=data.merge(temp[['doc_id','pretranslation']],how='left',on='doc_id').drop_duplicates('pretranslation').drop_duplicates('clean_msg')
# data=data[['class','processed']].drop_duplicates().rename(columns={'processed':'clean_msg'})
data['lang']=data['doc_id'].apply(lambda x: x[6])
data=data[data['clean_msg']!=''].reset_index(drop=True)
data

,doc_id,class,translated,clean_msg,pretranslation,lang
0,trec06c/data/000/007,spam,Dear person in charge (manager/finance): Hello...,dear person charge hello trading co ltd strong...,尊敬的负责人（经理／财务）：您好！ 我是深圳伟仕嘉贸易有公司：兴办贸易、物资供销，实力雄厚；...,c
1,trec06c/data/000/014,spam,Hello! Nice to meet you. Our company is intere...,hello nice meet company interested cooperating...,"您好！很高兴认识您。我司有意与您们合作:可长久给您们带来 巨大的效益,另因合作项目的高度自动...",c
2,trec06c/data/000/005,spam,TO: Your company’s manager and finance manager...,manager finance manager hello trading co ltd b...,TO：贵公司经理、财务 您好！ 深圳市春洋贸易有限公司（东莞分公司）...,c
3,trec06c/data/000/013,spam,New remote-controlled airplanes are widely sol...,new airplanes widely sold many types toy airpl...,新型遥控飞机销路广 百元可办厂 玩具飞机品种繁多，但用微型小开关控制的新型能在空中自控飞翔的...,c
4,trec06c/data/000/021,spam,"Dear company (factory) manager, hello: Our com...",dear company factory manager hello company com...,尊敬的公司（工厂）经理负责人你好： 我公司是一家多年为外资企业代理进出口业务的公司，现有部分...,c
...,...,...,...,...,...,...
39240,trec06p/data/035/115,spam,Rent WITHOUT COMMISSION\r \r m. Avtozavodskaya...,rent without commission min foot blocks buildi...,"Аренда БЕЗ КОМИССИОННЫХ\r \r м. Автозаводская,...",p
39241,trec06p/data/058/203,ham,RIFFx~ WAVEfmt + + dataS~ ~y{yw{ zuvx}}ytmjhed...,aa aa tr pf en kn la qu oka pk oz gun nom mum ...,RIFFx~ WAVEfmt      +  +    dataS~ ...,p
39242,trec06p/data/014/167,spam,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...,p
39243,trec06p/data/025/231,spam,:*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆...,largest online community history born site abs...,\r :*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆...,p


In [ ]:
data.groupby('class').count()

,doc_id,translated,clean_msg,pretranslation,lang
class,,,,,
ham,30579,30579,30579,30579,30579
spam,8666,8666,8666,8666,8666


In [ ]:
# en_regex = re.compile('[^a-zA-Z ]')
# d = enchant.Dict("en_US") 
# STOPWORDS = stopwords.words('english')

In [ ]:
# def enPreprocess(string):
#     string=string.replace('\n',' ') #remove newline char
#     string=en_regex.sub('',string) #removes non alphabets
#     string=string.split()
#     string=[word.lower() for word in string if len(word)>1]
#     string=' '.join([word for word in string if d.check(word) and (word not in STOPWORDS)]) # split the string into list and check each word if it is in the english dictionary and longer than 1 alphabet
#     return string

In [ ]:
# data['clean_msg']=data['translated'].apply(lambda x: enPreprocess(x))
# data.to_pickle('lowercase words.pkl')

In [ ]:

X = data.drop(columns='class')
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=.2,random_state=42)

In [ ]:
y_train.value_counts()

ham     19566
spam     5550
Name: class, dtype: int64

In [ ]:
y_train=pd.concat([y_train[y_train=='ham'].sample(5528),y_train[y_train=='spam']]).sample(frac=1) #under sampling
# y_train=pd.concat([y_train[y_train=='ham'],y_train[y_train=='spam'].sample(24461,replace=True)]).sample(frac=1) #over sampling

In [ ]:
X_train=X_train.loc[y_train.index]

In [ ]:
X_train_e=X_train[X_train['lang']=='p']['clean_msg']
X_val_e=X_val[X_val['lang']=='p']['clean_msg']
X_test_e=X_test[X_test['lang']=='p']['clean_msg']
y_train_e=y_train.loc[X_train_e.index]
y_val_e=y_val.loc[X_val_e.index]
y_test_e=y_test.loc[X_test_e.index]

X_train_c=X_train[X_train['lang']=='c']['pretranslation']
X_val_c=X_val[X_val['lang']=='c']['pretranslation']
X_test_c=X_test[X_test['lang']=='c']['pretranslation']
y_train_c=y_train.loc[X_train_c.index]
y_val_c=y_val.loc[X_val_c.index]
y_test_c=y_test.loc[X_test_c.index]

X_train=X_train['clean_msg']
X_val=X_val['clean_msg']
X_test=X_test['clean_msg']

In [ ]:
vect = CountVectorizer(stop_words='english')
X_train_dtm = vect.fit_transform(X_train)
X_val_dtm = vect.transform(X_val)
X_test_dtm= vect.transform(X_test)



In [ ]:
tfidvect=TfidfTransformer(smooth_idf=1)
# X_train_dtm = tfidvect.fit_transform(X_train_dtm)
# X_val_dtm = tfidvect.transform(X_val)
# X_test_dtm= tfidvect.transform(X_test)

In [ ]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_val_dtm)

print('translated dataset Multinomial Naive Bayes ')
print('acc',metrics.accuracy_score(y_val, y_pred_class))
print('f1',f1_score(y_val.to_list(), y_pred_class,pos_label="spam"))
# metrics.confusion_matrix(y_test, y_pred_class)

translated dataset Multinomial Naive Bayes 
acc 0.9280254777070064
f1 0.8371757925072046


In [ ]:
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_val_dtm)
y_pred_prob = logreg.predict_proba(X_val_dtm)[:, 1]

print('translated dataset log regression ')
print('acc',metrics.accuracy_score(y_val, y_pred_class))
print('validation f1',f1_score(y_val.to_list(), y_pred_class,pos_label="spam"))

translated dataset log regression 
acc 0.9503184713375796
validation f1 0.8963455149501661


In [ ]:
a=[]
b=[]
for i in np.arange(0.7,.85,0.001):
    a.append(i)
    c=pd.Series(y_pred_prob).apply(lambda x: 'spam' if x>i else 'ham')
    b.append(f1_score(y_val.to_list(),c,pos_label="spam"))


In [ ]:
thresh_hold_table=pd.DataFrame({'threshhold value':a,'f1_score':b}).sort_values('f1_score',ascending=False).head(10)
thresh_hold_table

,threshhold value,f1_score
96,0.796,0.919573
86,0.786,0.919475
85,0.785,0.919475
84,0.784,0.919475
95,0.795,0.919246
81,0.781,0.919206
97,0.797,0.919188
83,0.783,0.919149
82,0.782,0.919149
98,0.798,0.919131


In [ ]:
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
opt_predicted=pd.Series(y_pred_prob).apply(lambda x: 'spam' if x>thresh_hold_table.iloc[0,0] else 'ham')
print('test f1',f1_score(y_test.to_list(), opt_predicted,pos_label="spam"))

test f1 0.9172672239838571


In [ ]:
sample=pd.concat([data[data['class']=='ham'].sample(8666),data[data['class']=='spam']])

In [ ]:
ham_count=[]
spam_count=[]
word_list=[]
temp=pd.DataFrame({'weights':logreg.coef_[0]}).reset_index().merge(pd.DataFrame({'index':vect.vocabulary_.values(),'word':vect.vocabulary_.keys()}),how='left',on='index')
for index,(word,weights) in temp[temp['weights'].apply(lambda x:abs(x)>1)][['word','weights']].iterrows():
    counter=sample[sample['pretranslation'].apply(lambda x: word in x)]['class'].value_counts()
    try:
        ham_count.append(counter['ham'])
    except:
        ham_count.append(0)
    try:
        spam_count.append(counter['spam'])
    except:
        spam_count.append(0)
    word_list.append(word)

word_ham_spam_counter=pd.DataFrame({'word':word_list,'ham_count':ham_count,'spam_count':spam_count})

temp=temp.merge(word_ham_spam_counter,how='left',on='word')
temp[temp['ham_count']>5].sort_values('weights',ascending=True)


,index,weights,word,ham_count,spam_count
25092,25092,-2.503356,thanks,231.0,23.0
25361,25361,-2.233811,title,66.0,69.0
2732,2732,-1.398841,board,887.0,105.0
3262,3262,-1.391454,build,223.0,77.0
19261,19261,-1.315924,produced,35.0,8.0
27728,27728,-1.315301,wrote,942.0,12.0
18341,18341,-1.306825,pine,90.0,43.0
14630,14630,-1.303556,list,833.0,213.0
19230,19230,-1.296658,problem,859.0,147.0
16717,16717,-1.219624,normal,152.0,96.0


# 1 model for each language

In [ ]:
vect = CountVectorizer(stop_words='english')
X_train_e_dtm = vect.fit_transform(X_train_e)
X_test_e_dtm= vect.transform(X_test_e)

In [ ]:
import re
from TCSP import read_stopwords_list
cn_word=re.compile("[\u4e00-\u9FFF]")
X_train_c=X_train_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))
X_test_c=X_test_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))

In [ ]:
vect = CountVectorizer(tokenizer=lambda txt:[*txt],stop_words=read_stopwords_list())
X_train_c_dtm = vect.fit_transform(X_train_c)
X_test_c_dtm= vect.transform(X_test_c)

C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w', '介', '代', '令', '似', '位', '余', '作', '例', '便', '候', '值', '假', '僅', '儘', '儻', '先', '免', '全', '兩', '具', '兼', '出', '切', '前', '加', '反', '受', '句', '叮', '否', '呼', '哧', '唯', '唷', '問', '啪', '啷', '喔', '單', '喻', '噠', '固', '基', '外', '天', '夫', '奈', '妨', '妳', '始', '孰', '家', '少', '尚', '巧', '幸', '庶', '徒', '循', '怕', '恰', '悉', '惟', '慢', '成', '截', '抑', '拘', '接', '換', '料', '方', '旁', '旦', '曰', '期', '果', '根', '極', '樣', '次', '止', '正', '步', '死', '毋', '沒', '況', '消', '漫', '烏', '然', '特', '猶', '獨', '甚', '登', '直', '相', 

In [ ]:

nb = MultinomialNB()
nb.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = nb.predict(X_test_e_dtm)


nb = MultinomialNB()
nb.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = nb.predict(X_test_c_dtm)

In [ ]:
print('2 model combined Multinomial Naive Bayes ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined Multinomial Naive Bayes 
acc 0.9401197604790419
f1 0.8722131593257205


In [ ]:

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = logreg.predict(X_test_e_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = logreg.predict(X_test_c_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

In [ ]:

print('2 model combined log regression ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined log regression 
acc 0.9603771181042171
f1 0.9151432469304229


In [ ]:
ham_count=[]
spam_count=[]
word_list=[]
temp=pd.DataFrame({'weights':logreg.coef_[0]}).reset_index().merge(pd.DataFrame({'index':vect.vocabulary_.values(),'word':vect.vocabulary_.keys()}),how='left',on='index')
for index,(word,weights) in temp[temp['weights'].apply(lambda x:abs(x)>1)][['word','weights']].iterrows():
    counter=sample[sample['pretranslation'].apply(lambda x: word in x)]['class'].value_counts()
    try:
        ham_count.append(counter['ham'])
    except:
        ham_count.append(0)
    try:
        spam_count.append(counter['spam'])
    except:
        spam_count.append(0)
    word_list.append(word)

word_ham_spam_counter=pd.DataFrame({'word':word_list,'ham_count':ham_count,'spam_count':spam_count})

temp=temp.merge(word_ham_spam_counter,how='left',on='word')
temp[temp['ham_count']>5].sort_values('weights',ascending=True)

,index,weights,word,ham_count,spam_count
1251,1251,1.041424,息,526,1591
674,674,1.041125,图,227,644
3449,3449,0.940270,详,124,1289
3751,3751,0.785325,邮,143,2291
99,99,0.768213,件,915,2254
...,...,...,...,...,...
772,772,-0.863598,太,1450,424
2582,2582,-0.864265,研,854,651
1511,1511,-0.918331,掩,56,6
1597,1597,-0.952178,政,198,608


In [ ]:
temp_c[temp_c['ham_count']>56].sort_values('weights',ascending=True)

,index,weights,word,ham_count,spam_count
4058,4058,-1.080577,题,1817,963
1597,1597,-0.952178,政,198,608
2582,2582,-0.864265,研,854,651
772,772,-0.863598,太,1450,424
3640,3640,-0.857633,较,1105,855
...,...,...,...,...,...
99,99,0.768213,件,915,2254
3751,3751,0.785325,邮,143,2291
3449,3449,0.940270,详,124,1289
674,674,1.041125,图,227,644


In [ ]:
sample[sample['pretranslation'].apply(lambda x: '免'  in x)]['class'].value_counts()

spam    1540
ham      230
Name: class, dtype: int64